# 6.2. 图像卷积

![](../img/6_11.png)

## 6.2.1. 互相关运算
严格来说，卷积层是个错误的叫法，因为它所表达的运算其实是**互相关运算**（cross-correlation），而不是卷积运算。

根据 6.1节中的描述，在卷积层中，输入张量和核张量通过互相关运算产生输出张量。

![](../img/6_1.png)


In [1]:
"""
接下来，我们在corr2d函数中实现如上过程，该函数接受输入张量X和卷积核张量K，并返回输出张量Y。
"""
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X,K):  #@save
    """"""
    h,w=K.shape
    Y=torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]=(X[i:i+h,j:j+w] * K).sum()
    return Y

# 由上图的输入张量X和卷积核张量K，我们来验证上述二维互相关运算的输出。
X=torch.tensor([[0.0,1.0,2.0],
                [3.0,4.0,5.0],
                [6.0,7.0,8.0]])

K=torch.tensor([[0.0,1.0],
                [2.0,3.0]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

## 6.2.2. 卷积层
卷积层对输入和卷积核权重进行互相关运算，并在添加标量偏置之后产生输出。所以，卷积层中的两个被训练的参数是**卷积核权重**和**标量偏置**。就像我们之前随机初始化全连接层一样，在训练基于卷积层的模型时，我们也随机初始化卷积核权重。

**核矩阵的大小**是超参数。

In [ ]:
"""
基于上面定义的corr2d函数实现二维卷积层。
在__init__构造函数中，将weight和bias声明为两个模型参数。
"""
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight=nn.Parameter(torch.rand(kernel_size))
        self.bias=nn.Parameter(torch.zeros(1))

    # 前向传播函数调用corr2d函数并添加偏置。
    def forward(self,x):
        return corr2d(x,self.weight)+self.bias


## 6.2.3. 图像中目标的边缘检测

In [7]:
"""
如下是卷积层的一个简单应用：通过找到像素变化的位置，来检测图像中不同颜色的边缘。

首先，我们构造一个6x8像素的黑白图像。中间四列为黑色（0），其余像素为白色（1）。
"""
X=torch.ones((6,8))
X[:,2:6]=0  # 将编号为2～6列的元素置为0（黑色）
"""
X
tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])
"""

"""
接下来，我们构造一个高度为1、宽度为2的卷积核K。当进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零
"""
K=torch.tensor([[1.0,-1.0]])
"""
现在，我们对参数X（输入）和K（卷积核）执行互相关运算。

如下所示，输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘，其他情况的输出为。
"""
Y=corr2d(X,K)
"""
Y
tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])
"""

"""
现在我们将输入的二维图像转置，再进行如上的互相关运算。 其输出如下，之前检测到的垂直边缘消失了。 不出所料，这个卷积核K只可以检测垂直边缘，无法检测水平边缘。
"""
corr2d(X.t(),K)
"""
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
"""

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 6.2.4. 学习卷积核
当有了更复杂数值的卷积核，或者连续的卷积层时，我们不可能手动设计滤波器。那么我们是否可以学习由X生成Y的卷积核呢？

In [11]:
"""
我们先构造一个卷积层，并将其卷积核初始化为随机张量。

接下来，在每次迭代中，我们比较Y与卷积层输出的平方误差，然后计算梯度来更新卷积核。

为了简单起见，我们在此使用内置的二维卷积层，并忽略偏置。
"""
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X=X.reshape((1,1,6,8))
Y=Y.reshape((1,1,6,7))
lr=3e-2  # 学习率：0.03

for i in range(10):
    Y_hat=conv2d(X)
    l=(Y_hat-Y) **2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr*conv2d.weight.grad
    if (i+1) % 2==0:
        print(f'epoch:{i+1}, loss:{l.sum():.3f}')

tensor([[[[1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.],
          [1., 1., 0., 0., 0., 0., 1., 1.]]]]) tensor([[[[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
          [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]]])
epoch:2, loss:12.703
epoch:4, loss:3.935
epoch:6, loss:1.399
epoch:8, loss:0.537
epoch:10, loss:0.214


In [9]:
# 在10次迭代之后，误差已经降到足够低。现在我们来看看我们所学的卷积核的权重张量。学习到的卷积核权重非常接近我们之前定义的卷积核K。
conv2d.weight.data.reshape((1,2))

tensor([[ 1.0207, -0.9661]])

## 6.2.5. 互相关和卷积
为了得到正式的卷积运算输出，我们需要执行[6.1节](./L6_1_从全连接层到卷积.ipynb)中定义的严格卷积运算，而不是互相关运算。

为了与深度学习文献中的标准术语保持一致，我们将继续把“互相关运算”称为卷积运算，尽管严格地说，它们略有不同。 此外，对于卷积核张量上的权重，我们称其为**元素**。

## 6.2.6. 特征映射和感受野
上图中输出的卷积层有时被称为**特征映射（feature map）**，因为它可以被视为一个输入映射到下一层的空间维度的转换器。

在卷积神经网络中，对于某一层的任意元素x，其**感受野**（receptive field）是指在前向传播期间可能影响x计算的所有元素（来自所有先前层）。

请注意，感受野可能大于输入的实际大小。让我们用上图为例来解释感受野：给定$2 \times 2$卷积核，阴影输出元素值$19$的感受野是输入阴影部分的四个元素。假设之前输出为$\mathbf{Y}$，其大小为$2 \times 2$，现在我们在其后附加一个卷积层，该卷积层以$\mathbf{Y}$为输入，输出单个元素$z$。在这种情况下，$\mathbf{Y}$上的$z$的感受野包括$\mathbf{Y}$的所有四个元素，而输入的感受野包括最初所有九个输入元素。因此，当一个特征图中的任意元素需要检测更广区域的输入特征时，我们可以构建一个更深的网络。

## 6.2.7. 小结
- 二维卷积层的核心计算是二维互相关运算。最简单的形式是，对二维输入数据和卷积核执行互相关操作，然后添加一个偏置。
- 我们可以设计一个卷积核来**检测图像的边缘**。
- 我们可以从数据中学习卷积核的参数。
- 学习卷积核时，无论用严格卷积运算或互相关运算，卷积层的输出不会受太大影响。
- 当需要检测输入特征中更广区域时，我们可以构建一个更深的卷积网络。